In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Kaggle dataset

In [ ]:
# directory path
train = ''
train_img_dir = ''
test =''
test_img_dir =''
valid =''
val_img_dir =''

In [ ]:
# Wczytywanie obrazów i etykiet treningowych
train_images = []
train_labels = []
for idx, row in train.iterrows():
    img_path = os.path.join(train_img_dir, row['FILENAME'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))
    train_images.append(img)
    train_labels.append(row['IDENTITY'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Wczytywanie obrazów i etykiet testowych
test_images = []
test_labels = []
for idx, row in test.iterrows():
    img_path = os.path.join(test_img_dir, row['FILENAME'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))
    test_images.append(img)
    test_labels.append(row['IDENTITY'])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Wczytywanie obrazów i etykiet validacyjnych
val_images = []
val_labels = []
for idx, row in valid.iterrows():
    img_path = os.path.join(val_img_dir, row['FILENAME'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))
    val_images.append(img)
    val_labels.append(row['IDENTITY'])

val_images = np.array(val_images)
val_labels = np.array(val_labels)


# Mnist dataset

In [ ]:
# Załadowanie danych MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape danych
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalizacja danych
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Konwersja etykiet do postaci one-hot
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Podział danych na zbiór treningowy i walidacyjny
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

train_images, train_labels = x_train, y_train


# Experiment

In [ ]:

# Przykładowe funkcje do przetwarzania wstępnego obrazów
def scale_images(images):
    # Implementacja skalowania obrazów
    scaled_images = images * 255  # Przykładowe skalowanie
    return scaled_images

def normalize_images(images):
    # Implementacja normalizacji obrazów
    normalized_images = images / 255  # Przykładowa normalizacja
    return normalized_images

def segment_images(images):
    # Implementacja segmentacji obrazów
    segmented_images = images  # Przykładowa segmentacja (brak zmiany)
    return segmented_images

# Funkcja do tworzenia modelu OCR
def create_ocr_model(num_layers=2, filter_size=3, activation='relu'):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(filter_size, filter_size), activation=activation, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    for _ in range(num_layers - 1):
        model.add(Conv2D(32, kernel_size=(filter_size, filter_size), activation=activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Funkcja do przetwarzania wstępnego obrazów
def preprocess_images(images, scaling=False, normalization=False, segmentation=False):
    processed_images = images.copy()
    
    if scaling:
        processed_images = scale_images(processed_images)
    
    if normalization:
        processed_images = normalize_images(processed_images)
    
    if segmentation:
        processed_images = segment_images(processed_images)
    
    return processed_images

# Funkcja do oceny dokładności modelu OCR
def evaluate_ocr_model(images, labels):
    model = create_ocr_model()
    model.fit(images, labels, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    
    # Ewaluacja modelu
    y_pred = model.predict(images)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(labels, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    report = classification_report(y_true_classes, y_pred_classes)

    return accuracy, report


# Funkcja do przeprowadzenia eksperymentu z doborem hiperparametrów
def run_hyperparameter_experiment(num_layers, filter_size, activation):
    model = create_ocr_model(num_layers=num_layers, filter_size=filter_size, activation=activation)

    # Trening modelu
    history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val), verbose=0)

    # Ewaluacja modelu
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    report = classification_report(y_true_classes, y_pred_classes)

    return accuracy, report


# Eksperymenty z przetwarzaniem wstępnym i doborem hiperparametrów
def experiment():
    
    # Eksperyment z przetwarzaniem wstępnym
    def preprocess_experiment():
        results = []

        # Eksperyment 1: Przetwarzanie wstępne obrazów - skalowanie i normalizacja
        processed_images = preprocess_images(train_images, scaling=True, normalization=True)
        accuracy_1, report_1 = evaluate_ocr_model(processed_images, train_labels)
        results.append({'preprocessing': 'Scaling + Normalization', 'accuracy': accuracy_1, 'report': report_1})

        # Eksperyment 2: Przetwarzanie wstępne obrazów - segmentacja
        processed_images = preprocess_images(train_images, segmentation=True)
        accuracy_2, report_2 = evaluate_ocr_model(processed_images, train_labels)
        results.append({'preprocessing': 'Segmentation', 'accuracy': accuracy_2, 'report': report_2})

        # Eksperyment 3: Przetwarzanie wstępne obrazów - skalowanie, normalizacja i segmentacja
        processed_images = preprocess_images(train_images, scaling=True, normalization=True, segmentation=True)
        accuracy_3, report_3 = evaluate_ocr_model(processed_images, train_labels)
        results.append({'preprocessing': 'Scaling + Normalization + Segmentation', 'accuracy': accuracy_3, 'report': report_3})

        # Wyświetlenie wyników eksperymentów przetwarzania wstępnego
        for result in results:
            print(f"Preprocessing Techniques: {result['preprocessing']}")
            print(f"Accuracy: {result['accuracy']}")
            print("Classification Report:")
            print(result['report'])
            print("\n" + "="*50 + "\n")


    # Eksperyment z doborem hiperparametrów
    def hyperparameter_experiment():
        hyperparameter_results = []

        # Eksperyment 1: Defaultowe hiperparametry
        accuracy, report = run_hyperparameter_experiment(num_layers=2, filter_size=3, activation='relu')
        hyperparameter_results.append({'hyperparameters': 'Default', 'accuracy': accuracy, 'report': report})

        # Eksperyment 2: Różna liczba warstw
        accuracy, report = run_hyperparameter_experiment(num_layers=3, filter_size=3, activation='relu')
        hyperparameter_results.append({'hyperparameters': '3 Layers', 'accuracy': accuracy, 'report': report})

        # Eksperyment 3: Różny rozmiar filtra
        accuracy, report = run_hyperparameter_experiment(num_layers=2, filter_size=5, activation='relu')
        hyperparameter_results.append({'hyperparameters': 'Filter Size 5x5', 'accuracy': accuracy, 'report': report})

        # Eksperyment 4: Różna funkcja aktywacji
        accuracy, report = run_hyperparameter_experiment(num_layers=2, filter_size=3, activation='tanh')
        hyperparameter_results.append({'hyperparameters': 'Tanh Activation', 'accuracy': accuracy, 'report': report})

        # Wyświetlenie wyników eksperymentów z doborem hiperparametrów
        for result in hyperparameter_results:
            print(f"Hyperparameters: {result['hyperparameters']}")
            print(f"Accuracy: {result['accuracy']}")
            print("Classification Report:")
            print(result['report'])
            print("\n" + "="*50 + "\n")

    preprocess_experiment()
    hyperparameter_experiment()

experiment()
